In [ ]:
import numpy as np
from numpy import array,loadtxt
import pandas as pd
from sklearn.model_selection import train_test_split,cross_val_score
import joblib
from sklearn import model_selection,metrics
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.pipeline import make_pipeline 
from mlxtend.classifier import StackingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split,cross_val_score,KFold,cross_validate
from sklearn.metrics import precision_score,accuracy_score,roc_auc_score,recall_score,confusion_matrix,f1_score,mean_absolute_error
from mlxtend.feature_selection import ColumnSelector
import matplotlib as mpl
from matplotlib import pyplot as plt
import copy
from alipy import ToolBox
import alipy
from alipy.query_strategy import QueryInstanceGraphDensity,QueryInstanceDensityWeighted, QueryInstanceUncertainty
import warnings
warnings.filterwarnings("ignore")

In [ ]:
def stmodel(num):
    sclf = RandomForestClassifier(oob_score=True,n_jobs=-1)
    sxgb = XGBClassifier(eval_metric=['logloss','auc','error'],max_depth=12,n_jobs=-1)
    sgnb = GaussianNB()
    pipe1 = make_pipeline(ColumnSelector(cols=range(num)),sclf)

    pipe2 = make_pipeline(ColumnSelector(cols=range(num)),sxgb)

    pipe3 = make_pipeline(ColumnSelector(cols=range(num)),sgnb)

    stack = StackingClassifier(classifiers=[pipe1,pipe2,pipe3], meta_classifier=LogisticRegression(solver="lbfgs",class_weight = 'balanced'))
    return stack

In [ ]:
X = np.loadtxt('reduction50.csv')
y = np.loadtxt('y_label.csv')

In [ ]:
# if you would like to run it quickly, please excute the following code:
n = 10 # The value of n is determined by you
X=X[0:40000:n,:]
y = y[0:40000:n]
print(X.shape,y.shape)

# LAL

In [ ]:
%%time
alibox = ToolBox(X=X, y=y, query_type='AllLabels', saving_path='...')
            
# Split data
alibox.split_AL(test_ratio=0.3, initial_label_rate=0.01, split_count=20)

# Use the default Logistic Regression classifier
#model = alibox.get_default_model()
model  = stmodel(50)

# The cost budget is 50 times querying
stopping_criterion = alibox.get_stopping_criterion('num_of_queries', 100)

def main_loop(alibox, strategy, round):
    # Get the data split of one fold experiment
    train_idx, test_idx, label_ind, unlab_ind = alibox.get_split(round)
    # Get intermediate results saver for one fold experiment
    saver = alibox.get_stateio(round)

    # Set initial performance point
    model.fit(X=X[label_ind.index, :], y=y[label_ind.index])
    pred = model.predict(X[test_idx, :])
    accuracy = alibox.calc_performance_metric(y_true=y[test_idx],
                                              y_pred=pred,
                                              performance_metric='accuracy_score')
    saver.set_initial_point(accuracy)
            
    # If the stopping criterion is simple, such as query 50 times. Use `for i in range(50):` is ok.
    while not stopping_criterion.is_stop():
    # Select a subset of Uind according to the query strategy
    # Passing model=None to use the default model for evaluating the committees' disagreement
        select_ind = strategy.select(label_index=label_ind, unlabel_index=unlab_ind, batch_size=1)
        label_ind.update(select_ind)
        unlab_ind.difference_update(select_ind)

        # Update model and calc performance according to the model you are using
        model.fit(X=X[label_ind.index, :], y=y[label_ind.index])
        pred = model.predict(X[test_idx, :])
        accuracy = alibox.calc_performance_metric(y_true=y[test_idx],
                                                y_pred=pred,
                                                performance_metric='accuracy_score')

        # Save intermediate results to file
        st = alibox.State(select_index=select_ind, performance=accuracy)
        saver.add_state(st)

        # Passing the current progress to stopping criterion object
        stopping_criterion.update_information(saver)
    # Reset the progress in stopping criterion object
    stopping_criterion.reset()
    return saver    
lal_result = []
#density_result = []

            
for round in range(10):
    train_idx, test_idx, label_ind, unlab_ind = alibox.get_split(round)
            
# Use pre-defined strategy
    lal = alibox.get_query_strategy(strategy_name="QueryInstanceLAL")
    lal.download_data()
    lal.train_selector_from_file(reg_est=5, reg_depth=3)
    lal_result.append(copy.deepcopy(main_loop(alibox, lal, round)))

analyser = alibox.get_experiment_analyser(x_axis='num_of_queries')
analyser.add_method(method_name='LAL', method_results=lal_result)
print(analyser)

# QUIRE

In [ ]:
%%time
aliboxq = ToolBox(X=X, y=y, query_type='AllLabels', saving_path='...')           
# Split data
aliboxq.split_AL(test_ratio=0.3, initial_label_rate=0.01, split_count=20)

model  = stmodel(50)
            
stopping_criterion = aliboxq.get_stopping_criterion('num_of_queries', 100)

def main_loop(aliboxq, strategy, round):
    # Get the data split of one fold experiment
    train_idx, test_idx, label_ind, unlab_ind = aliboxq.get_split(round)
    # Get intermediate results saver for one fold experiment
    saver = aliboxq.get_stateio(round)

    # Set initial performance point
    model.fit(X=X[label_ind.index, :], y=y[label_ind.index])
    pred = model.predict(X[test_idx, :])
    accuracy = aliboxq.calc_performance_metric(y_true=y[test_idx],
                                              y_pred=pred,
                                              performance_metric='accuracy_score')
    print('initial acc:',accuracy)
    saver.set_initial_point(accuracy)
            

    while not stopping_criterion.is_stop():
        # Select a subset of Uind according to the query strategy
        # Passing model=None to use the default model for evaluating the committees' disagreement
        select_ind = strategy.select(label_index=label_ind, unlabel_index=unlab_ind, batch_size=1)
        label_ind.update(select_ind)
        unlab_ind.difference_update(select_ind)

        # Update model and calc performance according to the model you are using
        model.fit(X=X[label_ind.index, :], y=y[label_ind.index])
        pred = model.predict(X[test_idx, :])
        accuracy = aliboxq.calc_performance_metric(y_true=y[test_idx],
                                                y_pred=pred,
                                                performance_metric='accuracy_score')
        
        # Save intermediate results to file
        st = aliboxq.State(select_index=select_ind, performance=accuracy)
        saver.add_state(st)

        # Passing the current progress to stopping criterion object
        stopping_criterion.update_information(saver)
    # Reset the progress in stopping criterion object
    stopping_criterion.reset()
    return saver    
quire_result = []
       
for round in range(5):
    train_idx, test_idx, label_ind, unlab_ind = aliboxq.get_split(round)          
    quire = aliboxq.get_query_strategy(strategy_name="QueryInstanceQUIRE",train_idx=train_idx, kernel='linear')
    quire_result.append(copy.deepcopy(main_loop(aliboxq,quire, round)))

analyserq = aliboxq.get_experiment_analyser(x_axis='num_of_queries')
analyserq.add_method(method_name='QUIRE', method_results=quire_result)
print(analyserq)
analyserq.plot_learning_curves(title='QUIRE', std_area=False)